In [1]:
import networkx as nx
from tqdm import tqdm
import random
from leafer import Leafer
import numpy as np
import pickle
import os
import glob

In [44]:
# G = nx.read_edgelist("./IsA_Graph2.edgelist", delimiter=" ", create_using=nx.DiGraph)
G = nx.read_edgelist("data/filtered_wnet/it_isa.edgelist", delimiter="\t", create_using=nx.DiGraph)

In [45]:
l = Leafer(G)
# iterator = l.leafs_generator()

In [46]:
train, test = l.split_train_test(
    generation_depth=0,  # до какого уровня в топ. сортировке идти
    p=0.1,  # вероятность что подходящий случай уйдет в тест
    p_divide_leafs=0.5,
    # вероятность что листья поделим пополам трейн-тест
    # а не засунем целый случай в трейн или в тест
    min_to_test_rate=0.3,
    # минимальное количество доли вершин которых не было в
    # трейне чтобы поделить пополам на трейн-тест
    # то есть если 6\10 вершин были трейне то значит все 10 в трейн
    # если 5\10 были в трейне, то значит оставшиеся можем кинуть в тест
    weights=[0.01, 0.49, 0.4, 0.05, 0.05],
    # веса в соответствии
    # один ребенок, только листья, не только листья
    # триплеты с 2 родителями, триплеты такие что мидл нода имеет
    # 1 ребенка
)

In [47]:
len(train), len(test)

(5784, 467)

In [22]:
ls = [elem for elem in test if elem["case"] == "only_leafs_divided"]
len(ls)

13

In [23]:
train_count = {}
for elem in train:
    if elem["case"] in train_count.keys():
        train_count[elem["case"]] += 1

    else:
        train_count[elem["case"]] = 1

test_count = {}
for elem in test:
    if elem["case"] in test_count.keys():
        test_count[elem["case"]] += 1

    else:
        test_count[elem["case"]] = 1

In [24]:
train_count, test_count

({'leafs_and_no_leafs': 106,
  'only_leafs_divided': 106,
  'only_leafs_all': 101,
  'simple_triplet_2parent': 376,
  'simple_triplet_grandparent': 160,
  'only_child_leaf': 374},
 {'leafs_and_no_leafs': 9,
  'only_leafs_all': 14,
  'only_leafs_divided': 13,
  'simple_triplet_2parent': 38,
  'simple_triplet_grandparent': 18,
  'only_child_leaf': 34})

In [25]:
test[:5]

[{'children': ['Gesteinskörnung.n.1',
   'Ballast.n.1',
   'Bimetall.n.1',
   'Leinwand.n.1',
   'Damast.n.1',
   'Denim.n.1',
   'Drell.n.1',
   'Boden.n.1',
   'Gummiband.n.1',
   'Elastomer.n.1',
   'Filz.n.1',
   'Füllung.n.1',
   'Leinen.n.1',
   'Präkursor.n.1',
   'Gestein.n.1',
   'Elastan.n.1',
   'Bildwirkerei.n.1',
   'Voile.n.1',
   'Wolle.n.1',
   'Frottee.n.1'],
  'parents': 'Stoff.n.1',
  'grandparents': None,
  'case': 'leafs_and_no_leafs'},
 {'children': ['Sprengstoff.n.1', 'Dünger.n.1', 'Herbizid.n.1', 'Edukt.n.1'],
  'parents': 'chemischer_Stoff.n.1',
  'grandparents': None,
  'case': 'leafs_and_no_leafs'},
 {'children': ['Kassiterit.n.1',
   'Zinnober.n.1',
   'Coltan.n.1',
   'Kryolith.n.1',
   'Fluorit.n.1',
   'Pyrit.n.1',
   'Molybdänit.n.1',
   'Quarz.n.1',
   'Granat.n.1'],
  'parents': 'Mineral.n.1',
  'grandparents': None,
  'case': 'leafs_and_no_leafs'},
 {'children': ['Hasardeur.n.1', 'Söldner.n.1'],
  'parents': 'Abenteurer.n.1',
  'grandparents': None,
 

In [34]:
num_leaks = 0
for vertex in l.collector.test_verteces:
    if vertex in l.collector.train_verteces:
        num_leaks += 1

In [23]:
num_leaks

0

In [24]:
num_leaks = 0
for vertex in l.collector.train_verteces:
    if vertex in l.collector.test_verteces:
        num_leaks += 1

In [25]:
num_leaks

0

In [24]:
lang = 'en'
name_train = "../babel_datasets/reweighted_wnet_train_" + lang + "_babel.pickle"
name_test = "../babel_datasets/reweighted_wnet_test_" + lang + "_babel.pickle"

with open(name_train, "wb") as handle:
    pickle.dump(train, handle)

with open(name_test, "wb") as handle:
    pickle.dump(test, handle)

In [41]:
all_data = glob.glob(os.path.join(".", "*isa.edgelist"))
all_data

['./fr_isa.edgelist',
 './it_isa.edgelist',
 './en_isa.edgelist',
 './de_isa.edgelist',
 './es_isa.edgelist',
 './ru_isa.edgelist']

In [50]:
global_train = []
global_test = []

all_data = glob.glob(os.path.join("data/filtered_wnet/", "*isa.edgelist"))
for path in tqdm(all_data):
    lang = path.replace("data/filtered_wnet/", "").replace("_isa.edgelist", "")

    G = nx.read_edgelist(path, delimiter="\t", create_using=nx.DiGraph)
    l = Leafer(G)
    if lang == 'en':
        train, test = l.split_train_test(
        generation_depth=1,  # до какого уровня в топ. сортировке идти
        p=0.05,  # вероятность что подходящий случай уйдет в тест
        p_divide_leafs=0.5,
        min_to_test_rate=0.3,
        weights=[0.01, 0.49, 0.4, 0.05, 0.05],
    )
    else:
        train, test = l.split_train_test(
        generation_depth=0,  # до какого уровня в топ. сортировке идти
        p=0.1,  # вероятность что подходящий случай уйдет в тест
        p_divide_leafs=0.5,
        min_to_test_rate=0.3,
        weights=[0.01, 0.49, 0.4, 0.05, 0.05],
    )
        
    print(lang, len(train), len(test))

    random.shuffle(train)
    random.shuffle(test)

    name_train = "../babel_datasets/filtered_wnet/train_" + lang + "_babel.pickle"
    name_test = "../babel_datasets/filtered_wnet/test_" + lang + "_babel.pickle"

    with open(name_train, "wb") as handle:
        pickle.dump(train, handle)

    with open(name_test, "wb") as handle:
        pickle.dump(test, handle)

    global_train.extend(train)
    global_test.extend(test)

name_train = "../babel_datasets/filtered_wnet/global_train_babel.pickle"
name_test = "../babel_datasets/filtered_wnet/global_test_babel.pickle"


random.shuffle(global_train)
with open(name_train, "wb") as handle:
    pickle.dump(global_train, handle)

with open(name_test, "wb") as handle:
    pickle.dump(global_test, handle)

  0%|          | 0/6 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [00:02<00:11,  2.37s/it]

fr 6458 503


 33%|███▎      | 2/6 [00:04<00:09,  2.29s/it]

it 5810 439


...

de 1256 97


 83%|████████▎ | 5/6 [00:17<00:03,  3.08s/it]

es 5202 376


100%|██████████| 6/6 [00:18<00:00,  3.07s/it]

ru 1972 186


In [52]:
name_train = "../babel_datasets/wnet_only/global_train_babel.pickle"
name_test = "../babel_datasets/wnet_only/global_test_babel.pickle"

with open(name_train, "wb") as handle:
    pickle.dump(global_train, handle)

with open(name_test, "wb") as handle:
    pickle.dump(global_test, handle)

In [48]:
len(global_test), len(global_train)

(1980, 38808)

In [49]:
max_len = 0
k = 0
for vert in global_train:
    cur_len = len(vert["children"])
    if cur_len > 50:
        k += 1
        max_len = cur_len
        max_i = vert

In [51]:
max_i

{'children': 'conditions_normales_de_température_et_de_pression.n.1',
 'parents': ['atmosphère.n.1', 'température_standard.n.1'],
 'grandparents': None,
 'case': 'simple_triplet_2parent'}

In [17]:
def predict_child_with_parent_and_grandparent(elem):
    """
    hyperhypenym: arthropod.n.01,
    hypernym: insect.n.01, hyponyms:
    (blackly)

    hyperhypenym: elem['grandparents'],
    hypernym: elem['parents'], hyponyms:
    elem['children']

    Fly is a hyponym for the word “insect".
    Predict hyponyms for the word “fly”. Answer:
    """

    # transformed_term = (
    #     "hyperhypenym: "
    #     + ", ".join(elem["grandparents"])
    #     + ", hypernym: "
    #     + elem["parents"]
    #     + ", hyponyms:"
    # )
    transformed_term = (
        ", ".join(elem["grandparents"])
        + " are hyponyms for the word '"
        + elem["parents"]
        + "'. Predict hyponyms for the word '"
        + elem["parents"]
        + "'. Answer:"
    )
    return transformed_term